In [1]:
from cmdstanpy import CmdStanModel
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import math
import json
from simulator import popn2_simple_data_morgan

In [2]:
model = CmdStanModel(stan_file='two_popn_number.stan')

In [5]:
seed=1111
n=5
N = [7000,7000,7000]
T = 100
m=2
length=1e6
L = [0.5,200]
d = popn2_simple_data_morgan(N,T,L,m,length,n,seed)
d['N_obs'] = len(d['y'])
d['length']=100*m
d['N'] = [x*2 for x in N]

In [6]:
sum(d['y'])/len(d['y'])

2.4551724137931035

In [8]:
def int_p_L_divide_l(N,u,v,t1,t2):
    cu = - (1/N + u/50)
    cv = - (1/N + v/50)
    a1 = np.exp(cu*t2)*(cu*t2-1)/cu**2 - np.exp(cu*t1)*(cu*t1-1)/cu**2
    a2 = np.exp(cv*t2)*(cv*t2-1)/cv**2 - np.exp(cv*t1)*(cv*t1-1)/cv**2
    return (1/(50*N))*(a1-a2)

def likelihood(k,N,u,v,t1,t2):
    lamb = int_p_L_divide_l(N,u,v,t1,t2)
    return np.exp(-lamb)*lamb**k/math.factorial(k)

In [12]:
int_p_L_divide_l(14000,0.5,2000,100,10000000)*200

2.0650544178645065

In [13]:
l = 0
for i in d['y']:
    l+=math.log(likelihood(i,14000,0.5,200,0,1000000000))
l

-1755.372578816046

In [14]:
v = model.variational(data=d,elbo_samples=1000,grad_samples=20)
v.variational_params_dict['T']

11:36:10 - cmdstanpy - INFO - Chain [1] start processing
11:36:10 - cmdstanpy - INFO - Chain [1] done processing


54.7306

In [15]:
v

CmdStanVB: model=two_popn_number['method=variational', 'grad_samples=20', 'elbo_samples=1000', 'adapt', 'engaged=1']
 csv_file:
	/var/folders/gm/vzcskl_s1wzfqy1p36xcp32h0000gn/T/tmp33db6azy/two_popn_numbertbrlsruv/two_popn_number-20241101113610.csv
 output_file:
	/var/folders/gm/vzcskl_s1wzfqy1p36xcp32h0000gn/T/tmp33db6azy/two_popn_numbertbrlsruv/two_popn_number-20241101113610_0-stdout.txt

In [16]:
model.optimize(data=d)

11:36:18 - cmdstanpy - INFO - Chain [1] start processing
11:36:18 - cmdstanpy - INFO - Chain [1] done processing


CmdStanMLE: model=two_popn_number['method=optimize']
 csv_file:
	/var/folders/gm/vzcskl_s1wzfqy1p36xcp32h0000gn/T/tmp33db6azy/two_popn_numberijx7pjye/two_popn_number-20241101113618.csv
 output_file:
	/var/folders/gm/vzcskl_s1wzfqy1p36xcp32h0000gn/T/tmp33db6azy/two_popn_numberijx7pjye/two_popn_number-20241101113618_0-stdout.txt

In [17]:
mcmc = model.sample(data=d)
mcmc.summary()

11:36:37 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

11:36:37 - cmdstanpy - INFO - CmdStan done processing.
11:36:37 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: poisson_lpmf: Rate parameter is nan, but must be nonnegative! (in 'two_popn_number.stan', line 44, column 8 to column 74)
	Exception: poisson_lpmf: Rate parameter is nan, but must be nonnegative! (in 'two_popn_number.stan', line 44, column 8 to column 74)
	Exception: gamma_lpdf: Random variable is inf, but must be positive finite! (in 'two_popn_number.stan', line 40, column 4 to column 20)
	Exception: poisson_lpmf: Rate parameter is nan, but must be nonnegative! (in 'two_popn_number.stan', line 44, column 8 to column 74)
	Exception: poisson_lpmf: Rate parameter is nan, but must be nonnegative! (in 'two_popn_number.stan', line 44, column 8 to column 74)
	Exception: poisson_lpmf: Rate parameter is nan, but must be nonnegative! (in 'two_popn_number.stan', line 44, column 8 to column 74)
Exception: poisson_lpmf: Rate parameter is nan, but must be nonnegative! 

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
lp__,98.6892,0.015716,0.716014,97.2384,98.9590,99.1744,2075.57,7773.68,1.00050
T,57.7755,0.195990,7.954370,44.9721,57.6116,71.1943,1647.19,6169.26,1.00074
